구글 colab을 이용하여 실습을 진행하였다.

로지스틱 회귀에 사용 할 파일을 업로드하여 불러오도록 하겠다.

--------------------------------------------------------

In [1]:
#파일 업로드하기
from google.colab import files
myfile = files.upload()

Saving fruit.csv to fruit.csv


fruit.csv(application/vnd.ms-excel) - 5275 bytes, last modified: 2021. 11. 10. - 100% done
Saving fruit.csv to fruit.csv
<br> 업로드 됨

In [2]:
#io와 pandas 모듈 import
import io
import pandas as pd

코랩에서는 io모듈을 import 하지 않은 상태에서 pandas를 사용할 경우 오류 발생한다.

In [3]:
#pd.read_csv로 csv파일 불러오기
fruit = pd.read_csv(io.BytesIO(myfile['fruit.csv']))
fruit.head()

,Species,Weight,Length,Diagonal,Height,Width
0,apple,242.0,25.4,30.0,11.5200,4.0200
1,apple,290.0,26.3,31.2,12.4800,4.3056
2,apple,340.0,26.5,31.1,12.3778,4.6961
3,apple,363.0,29.0,33.5,12.7300,4.4555
4,apple,430.0,29.0,34.0,12.4440,5.1340


코랩에 업로드하여 파일 읽어오기 완료

-------------------------------------

In [4]:
# 주피터 노트북에서 진행한다면
#import pandas as pd
#fruit = pd.read_csv('./fruit.csv'))
#fruit.head()

In [5]:
fruit_input = fruit[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fruit_target = fruit['Species'].to_numpy()

종을 타깃으로 잡고, 나머지 열 제목들을 특성으로 잡는다.
<br>
- 데이터 프레임에서 여러 열을 선택하면 새로운 데이터 프레임으로 변환됨
- 이를 넘파이 배열로 바꿔서 저장

In [6]:
# 어떤 종류의 과일이 있는지 고유 값 추출
print(pd.unique(fruit['Species']))

['apple' 'pear' 'banana' 'orange' 'grape' 'melon' 'plum']


In [7]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fruit_input,fruit_target,random_state=42)

In [8]:
# 훈련 세트와 테스트 세트 표준화 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

훈련세트의 통계 값으로 테스트 세트를 변환

### K- 최근접 이웃의 다중 분류

In [9]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled,train_target))
print(kn.score(test_scaled,test_target))

0.8823529411764706
0.85


In [10]:
print(kn.classes_) # 모델이 데이터로부터 학습한 값

['apple' 'banana' 'grape' 'melon' 'orange' 'pear' 'plum']


In [11]:
print(kn.predict(test_scaled[:5])) # 5개의 샘플을 뽑아 최근접 알고리즘으로 확인

['grape' 'plum' 'melon' 'grape' 'grape']


In [12]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=4))

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     1.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.     0.3333 0.    ]
 [0.     0.     0.6667 0.     0.     0.3333 0.    ]]


확률 출력을 위해 predict_proba 메소드 사용.<br>
사이킷 런의 대부분의 모델들은 이 메소드 제공.
<br>확률 계산을 위해 자원이 너무 많이 필요한 모델은 제공 안해주는 경우도 있다.
<br><br>
다섯개의 샘플이 행으로 나와있고 7개의 과일에 대한 확률이 나와있다.<br>
['apple' 'banana' 'grape' 'melon' 'orange' 'pear' 'plum']<br>
[0 0 1 0 0 0 0]
=> 세번째 클래스(grape)에 대한 확률이 1이다.

K-최근접 이웃의 단점은 3개의 샘플만 사용했기 때문에 확률이 0/3, 1/3, 2/3, 3/3밖에 없다.

## 로지스틱 회귀
대표적인 _분류 알고리즘_<br><br>
**z = a*무게 + b*길이 +c*대각선 + d*높이 + e*두께 + f** 
<br>a,b,c,d,e,f는 가중치 혹은 계수<br><br>
z가 확률이 되려면 0~1 사이 값이 되어야한다.
- 시그모이드 함수(로지스틱 함수) 사용

In [13]:
# 로지스티 회귀(이진 분류)
apple_plum_indexes = (train_target == 'apple') | (train_target == 'plum')
train_apple_plum = train_scaled[apple_plum_indexes]
target_apple_plum = train_target[apple_plum_indexes]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_apple_plum,target_apple_plum)

print(lr.predict(train_apple_plum[:5]))

['apple' 'plum' 'apple' 'apple' 'apple']


apple_plum_indexes : 전체 모델 중 apple과 plum만 뽑아낸 인덱스

In [14]:
print(lr.predict_proba(train_apple_plum[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


이진 분류이기 때문에 첫번째 인덱스가 음성(0) 클래스의 확률, 두번째 인덱스가 양성 클래스(1)의 확률이다.

In [15]:
print(lr.classes_)

['apple' 'plum']


확인 결과 사과가 0, 자두가 1이다.<br>
사이킷런은 타깃값을 알페벳 순으로 정렬하여 사용한다.

### 로지스틱 회귀 계수 확인
선형함수를 학습하기때문에 출력 가능하다.

In [16]:
print(lr.coef_,lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


특성이 다섯개이므로 다섯개의 가중치(기울기)가 나온다.<br>
z = -0.404 * 무게 + (-0.576)*길이 + (-0.663)*대각선 + (-1.013)*높이 + (-0.732)*두께 + (-2.161)

In [17]:
# z 값을 출력해주는 decision_function
decisions = lr.decision_function(train_apple_plum[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


앞에서 넣은 다섯개의 샘플의 z(결과) 값을 출력<br><br>
z 값을 시그모이드 함수에 통과시키면 확률을 얻을 수 있다.

In [18]:
# 사이파이의 expit() - np.exp()을 이용해 분수계산보다 편리하고 안전하다.
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


[[0.99759855 **0.00240145**]<br>
 [0.02735183 **0.97264817**]<br>
 [0.99486072 **0.00513928**]<br>
 [0.98584202 **0.01415798**]<br>
 [0.99767269 **0.00232731**]]<br>
동일함.<br><br>
decision_function은<br>
로지스틱 회귀가 이진 분류일 때는 사실상 양성 클래스의 z값만 계산을 하는 것.<br>
확률을 출력할 때는 양성 클래스의 확률을 출력하는 것.<br>(1에서 결과값을 빼면 음성 클래스의 값이 나오긴 함)

<br>

## 로지스틱 회귀(다중 분류)
여러개의 클래스가 있는 경우

In [19]:
lr = LogisticRegression(C=20,max_iter=1000)
lr.fit(train_scaled, train_target)

LogisticRegression(C=20, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**max_iter**은 반복횟수 지정 매개변수이다. 기본 값은 100<br><br>
로지스틱 회귀는 기본적으로 계수의 제곱을 규제한다.(L2 규제)<br>
선형 회귀의 릿지 회귀에서는 alpha 매개변수로 규제의 양을 조절했다. alpha가 커지면 규제도 커지는 식으로..<br>
LogisticRegression에서 규제의 양 을 조절하는 매개변수는 **C**이다.<br>
하지만 alpha와 반대로 C의 값이 작아지면 규제가 커진다.<br><br>

 LogisticRegression(C=20,max_iter=1000)<br>
→ 규제는 완화하고, 반복횟수는 늘린 것.

In [20]:
# 분류의 score은 정확도이다.
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [21]:
# 테스트 세트의 5개의 샘플에 대한 예측
print(lr.predict(test_scaled[:5]))

['grape' 'plum' 'melon' 'pear' 'grape']


In [22]:
# 테스트 세트의 5개의 샘플에 대한 예측 확률
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3)) # 소수점 네번째에서 반올림

[[0.    0.003 0.841 0.    0.014 0.136 0.007]
 [0.    0.    0.044 0.    0.003 0.007 0.946]
 [0.    0.    0.034 0.935 0.    0.015 0.016]
 [0.011 0.076 0.306 0.007 0.034 0.567 0.   ]
 [0.    0.001 0.904 0.002 0.    0.089 0.002]]


5개의 행에 대한 7개의 확률<br>
[[0.    0.003 **0.841** 0.    0.014 0.136 0.007]<br>
 [0.    0.    0.044 0.    0.003 0.007 **0.946**]<br>
 [0.    0.    0.034 **0.935** 0.    0.015 0.016]<br>
 [0.011 0.076 0.306 0.007 0.034 **0.567** 0.   ]<br>
 [0.    0.001 **0.904** 0.002 0.    0.089 0.002]]

In [23]:
print(lr.classes_)

['apple' 'banana' 'grape' 'melon' 'orange' 'pear' 'plum']


In [24]:
# coef_ : 기울기
# intercept_ : 절편
print(lr.coef_.shape,lr.intercept_.shape)

(7, 5) (7,)


7개의 행과 5개의 열이 있는 행렬<br><br>
5개 : 특성과 곱해지는 계수<br>

7개 : 클래스마다 선형 함수가 하나씩 만들어짐. 즉, z값이 클래스마다 만들어지는 것<br><br>
7개의 선형 함수를 만들어서 샘플에 대해서 7개의 함수를 통과시켜서 나오는 값 중 가장 큰 값이 예측 클래스가 되는 것.<br>
각 클래스를 제외하고는 모두 <u>이진 분류</u>처럼 구한다. <br>
[0. 0.003 **0.841** 0. 0.014 0.136 0.007]를 구하는 경우에는
세번째 클래스는 양성, 나머지 클래스는 모두 음성으로 두고 값을 계산해서 나온 값이 확률을 구하는데 쓰인다.<br><br>

7개의 z를 어떻게 확률 변수로 바꿀 수 있을까?<br>
z를 모두 시그모이드로 바꾸면 7개의 시그모이드의 합이 1이 되어야 한다.<br>
→ 보통 1이 안됨...
<br>
그래서 사용하는 것이 
### 소프트맥스 함수

In [26]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision,decimals=2))

[[ -6.5   -0.63   5.16  -2.73   1.03   3.34   0.33]
 [-10.86  -4.26   4.77  -2.4    1.93   2.98   7.84]
 [ -4.34  -3.87   3.17   6.49  -6.23   2.36   2.42]
 [ -0.68   1.26   2.65  -1.19   0.45   3.26  -5.75]
 [ -6.4   -0.71   5.82  -0.11  -1.99   3.5   -0.11]]


소프트맥스 함수는 z값을 지수함수(e)에 적용한다.<br>
[ -6.5&emsp; -0.63&emsp;5.16&emsp;-2.73 &emsp;1.03&emsp;3.34&emsp;0.33]<br>
z0&emsp;&emsp;z1 &emsp;&emsp; z2 &emsp;&emsp;z3&emsp;&emsp;z4 &emsp;&emsp;z5&emsp;&emsp;z6
<br><br>
s0 = (e^z0)/e_sum <br>
s1 = (e^z1)/e_sum <br>
...<br>
s0 + s1 + ... + s6 = 1

In [28]:
from scipy.special import softmax

proba = softmax(decision, axis = 1)
print(np.round(proba,decimals=3))

[[0.    0.003 0.841 0.    0.014 0.136 0.007]
 [0.    0.    0.044 0.    0.003 0.007 0.946]
 [0.    0.    0.034 0.935 0.    0.015 0.016]
 [0.011 0.076 0.306 0.007 0.034 0.567 0.   ]
 [0.    0.001 0.904 0.002 0.    0.089 0.002]]


다중분류_LogisticRegression을 이용해 출력한 확률값과 동일한 확률값이 출력됨